#### NAME
    find - search for files in a directory hierarchy
#### SYNOPSIS
    find [-H] [-L] [-P] [-D debugopts] [-Olevel] [path...] [expression]

#### DESCRIPTION
    This manual page documents the GNU version of find.
    GNU find searches the directory tree rooted at each given file name by evaluating the given expression from left to right, according  to the rules of precedence (see section OPERATORS), until the outcome i  known (the left hand side is false for and operations, true  for or), at which point find moves on to the next file name.

    If you are using find in an environment where security is important (for example if you are using it to search directories that  are  writable  by other users), you should read the "Security Considerations" chapter of the findutils documentation,which is called Finding Files and comes with findutils.That document also includes a lot more detail and discussion than this manual page, so you may find it a more useful source of information.

#### OPTIONS
    The  -H, -L and -P options control the treatment of symbolic links.
    Command-line arguments following these are taken to be names of files or directories to be examined, 
    up to the first argument that begins with `-', or the argument  `('  or  `!'.
    
    That argument and any following arguments are taken to be the expression describing what is to be searched for.
    If no paths are given,the current directory is used.
    If no expression is given,the expression -print is used (but you should probably consider using -print0 instead, anyway).
       
    This manual page talks about `options' within the expression list.
    These options control the behaviour of find but are specified immediately after the last path name.  
    The five `real' options -H, -L, -P, -D and -O must appear before the first path name,if at all.  
    A double dash -- can also be used to signal that any remaining arguments are not options (though ensuring that all start point 
    begin with either `./' or `/' is generally safer if you use wildcards in the list  of  start points).

###### -P     
    Never follow symbolic links.This is the default behaviour.
    When find examines or prints information a file, and the file is a symbolic link, the information used 
    shall be taken from the properties of the symbolic link itself.

###### -L     
    Follow symbolic links.
    When find examines or prints information about files, the information used shall be taken from the properties of the file to which the link points, not from the link itself 
    (unless it is a broken symbolic link or find is unable to examine the file to which the link points).
    
    Use of this option implies -noleaf.
    If you later use the -P option, -noleaf will still be in effect. 
    If -L is in effect and find discovers a symbolic link to a subdirectory during its search, the subdirectory pointed to by the symbolic link will be searched.
    
     When the -L option is in effect, the -type predicate will always match against the type of the file that a symbolic link points to 
     rather than the link itself (unless the symbolic link is broken).
     Using -L causes the -lname and -ilname predicates always to return false.

###### -H     
    Do not follow symbolic links, except while processing the command line arguments.
    When find examines or prints information about files,the information used shall be taken from the properties of the symbolic link itself.
    
    The only exception to this behaviour is when a file specified on the command line is a symbolic link, and the link can be resolved.
    For that situation,the information used is taken from whatever the link points to (that is,the link is followed).
    The information about the link itself is used as a fallback if the file pointed to by the symbolic link cannot be examined.
    If -H is in effect and one of the paths specified on the command line is a symbolic link to a directory, the contents of that 
    directory will be examined (though of course -maxdepth 0 would prevent this).

If more than one of -H, -L and -P is specified, each overrides the others; the last one appearing on the command line takes effect.Since it is the default, the -P option should be considered to be in effect unless either -H or -L is specified.

GNU find frequently stats files during the processing of the command line itself, before any searching has begun.  These options also affect how those arguments are processed.Specifically, there are a number of tests that compare files listed on the command line against a file we are currently considering.In each case,the file specified on the command line will have been examined and some of its properties will have been saved.If the named file is in fact a symbolic link, and  the -P option is in effect (or if neither -H nor -L were specified), the information used for the comparison will be taken from the properties of the symbolic link.  Otherwise, it will be taken from the properties of the file the link points to.If find  cannot  follow the link (for example because it has insufficient privileges or the link points to a nonexistent file) the properties of the link itself will be used.

When the -H or -L options are in effect, any symbolic links listed as the argument of -newer will be dereferenced, and the timestamp will be taken from the file to which the symbolic link points.The same consideration applies to -newerXY, -anewer and -cnewer.

The -follow option has a similar effect to -L, though it takes effect at the point where it appears (that is, if -L is not used but -follow is, any symbolic links appearing after -follow on the command line will be dereferenced, and those before it will not).

###### -D debugoptions
    Print diagnostic information;this can be helpful to diagnose problems with why find is not doing what you want. 
    The list of debug options should be comma separated.
    Compatibility of the debug options is not guaranteed between releases of findutils.
    For a complete list of valid debug options, see the output of find  -D  help.
    Valid debug options include:
        help   Explain the debugging options
        
        tree   Show the expression tree in its original and optimised form.
        
        stat   Print messages as files are examined with the stat and lstat system calls.The find program tries to minimise such calls.
             
        opt    Prints diagnostic information relating to the optimisation of the expression tree;see the -O option.

        rates   Prints a summary indicating how often each predicate succeeded or failed.

###### -Olevel
    Enables query optimisation.
    The find program reorders tests to speed up execution while preserving the overall effect; 
    that is, predicates with side effects are not reordered relative to each other.
    The optimisations performed at each optimisation level are as follows.
    
    0      Equivalent to optimisation level 1.
    
    1      This is the default optimisation level and corresponds to the traditional behaviour.
           Expressions are reordered so that tests based only on the names of files 
           (for example -name and -regex) are performed first.
           
    2      Any -type or -xtype tests are performed after any tests based only on the names of files, 
           but before any tests that require information from the inode.
           On many modern versions of Unix, file types are  returned  by readdir() and so these predicates
           are faster to evaluate than predicates which need to stat the file first.
           
    3      At this optimisation level, the full cost-based query optimiser is enabled.
           The order of tests is modified so that cheap (i.e. fast) tests are performed first and more
           expensive ones are performed  later,if  necessary.
           Within each cost band, predicates are evaluated earlier or later according to whether 
           they are likely to succeed or not.  
           For -o, predicates which are likely to succeed are evaluated earlier, and for -a,  
           predicates which are likely to fail are evaluated earlier.

The cost-based optimiser has a fixed idea of how likely any given test is to succeed.
In some cases the probability takes account of the specific nature of the test (for example, -type f is assumed to be more likely to succeed than -type c).The cost-based optimiser is currently being evaluated.If it does not actually improve the performance of find, it will be removed again.  Conversely, optimisations that prove to be reliable, robust and effective may be enabled at lower optimisation levels over time.However, the default behaviour (i.e. optimisation level 1) will not be changed in the 4.3.x release series.The findutils test suite runs all the tests on find at each optimisationlevel and ensures that the result is the same.

#### EXPRESSIONS
    The expression is made up of 
    options (which affect overall operation rather than the processing of a specific file, and always return true), 
    tests (which return a true or false value), 
    and actions (which have side effects and return a true or false value), 
    all separated by operators.-and is assumed where the operator is omitted.

    If the expression contains no actions other than -prune, -print is performed on all files for which the expression is true.

#### A.OPTIONS
All options always return true.
Except for -daystart, -follow and -regextype, the options affect all tests, including tests specified before the option. This is because the options are processed when the command line is parsed,while the tests don't do anything until files are examined. The -daystart, -follow and -regextype options are different in this respect, and have an effect only on tests which appear later in the command line.Therefore, for clarity, it is best to place them at the beginning of the expression.A warning is issued if you don't do this.

###### -d     
    A synonym for -depth, for compatibility with FreeBSD, NetBSD, MacOS X and OpenBSD.

###### -daystart
    Measure times (for -amin, -atime, -cmin, -ctime, -mmin, and -mtime) from the beginning of today rather than from 24 hours ago.
    This option only affects tests which appear later on the command line.

###### -depth 
    Process each directory's contents before the directory itself.
    The -delete action also implies -depth.

###### -follow
    Deprecated; use the -L option instead.
    Dereference symbolic links.Implies -noleaf.
    The -follow option affects only those tests which appear after it on the command line.Unless the -H or -L option has been specified,
    the position of the -follow option changes the behaviour of the -newer predicate; 
    any files listed as the argument of -newer will be dereferenced if they are symbolic links.
    
    The same consideration applies to -newerXY, -anewer and -cnewer.
    Similarly,the -type predicate will always match against the type of the file that a symbolic link points to rather than the link itself.
    Using -follow causes the -lname and -ilname predicates always to return false.

######  -help, --help
    Print a summary of the command-line usage of find and exit.

###### -ignore_readdir_race
    Normally, find will emit an error message when it fails to stat a file.
    If you give this option and a file is deleted between the time find reads the name of the file from the directory and the time it
    to stat the file, no error message will be issued.
    This also applies to files or directories whose names are given on the command line.  
    This option takes effect at the ime the command line is read, which means that you cannot search one part of the filesystem with this option on and part of it with this option off 
    (if you need to do that, you will need to ssue two find commands instead, one with the option and one without it).

###### -noignore_readdir_race
    Turns off the effect of -ignore_readdir_race.

###### -maxdepth levels
    Descend at most levels (a non-negative integer) levels of directories below the command line arguments.
    -maxdepth 0 means only apply the tests and actions to the command line arguments.

###### -mindepth levels
    Do not apply any tests or actions at levels less than levels (a non-negative integer).
    -mindepth 1 means process all files except the command line arguments.

###### -mount 
    Don't descend directories on other filesystems.
    An alternate name for -xdev, for compatibility with some other versions of find.

###### -noleaf
    Do not optimize by assuming that directories contain 2 fewer subdirectories than their hard link count.  
    This option is needed when searching filesystems that do not follow the Unix directory-link convention,
    such as CD-ROM or MS-DOS filesystems or AFS volume mount points.
    
    Each directory on a normal Unix filesystem has at least 2 hard links: its name and its `.' entry.
    Additionally, its subdirectories (if any) each have a `..' entry linked to that directory.
    When find is examining a directory, after it has statted 2 fewer subdirectories than the directory's link count, 
    it knows that the rest of the entries in the directory are non-directories (`leaf' files in the directory tree).
    
    If only the files' names need to be examined, there is no need to stat them; this gives a significant increase in search speed.

###### -regextype type
    Changes the regular expression syntax understood by -regex and -iregex tests which occur later on the command line.
    Currently-implemented types are emacs(this is the default),  posix-awk,posix-basic, posix-egrep and posix-extended.

###### -version, --version
    Print the find version number and exit.

###### -warn, -nowarn
    Turn  warning  messages on or off.
    These warnings apply only to the command line usage, 
    not to any conditions that find might encounter when it searches directories.
    
    The default behaviour corresponds to -warn if standard input is a tty, and to -nowarn otherwise.

###### -xdev 
    Don't descend directories on other filesystems.

#### TESTS
    Some tests, for example -newerXY and -samefile,allow comparison between the file currently being examined and some reference file specified on the command line.When these tests are used,the interpretation of the reference file is determined by the options -H, -L and -P and any previous -follow, but the reference file is only examined once, at the time the command line is parsed.If the reference file cannot be examined (for example, the stat(2) system call fails for it), an error message is issued, and find exits with a nonzero status.